In [79]:
from collections import OrderedDict
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        return dx

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class TwoLayerNeuralNet():
    def __init__(self,input_size=3072,hidden_size=50,output_size=10,weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = np.random.randn(input_size,hidden_size) * weight_init_std
        self.params["B1"] = np.zeros(hidden_size)
        self.params["W2"] = np.random.randn(hidden_size,output_size) * weight_init_std
        self.params["B2"] = np.zeros(output_size)
        self.layers = OrderedDict()
        self.layers["Affine1"] =  Affine(self.params['W1'], self.params['B1'])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] =  Affine(self.params['W2'], self.params['B2'])
        self.lastLayer = SoftmaxWithLoss()
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)

    def gradient(self,x,t):
        self.loss(x,t)
        dout = self.lastLayer.backward(1) # 
        layers_tmp = list(self.layers.values())
        layers_tmp.reverse()
        for layer in layers_tmp:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['B1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['B2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads
    

In [ ]:
class ThreeLayerNeuralNet():
    def __init__(self,input_size=3072,hidden_size=50,output_size=10,weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = np.random.randn(input_size,hidden_size) * weight_init_std
        self.params["B1"] = np.zeros(hidden_size)
        self.params["W2"] = np.random.randn(hidden_size,output_size) * weight_init_std
        self.params["B2"] = np.zeros(output_size)
        self.layers = OrderedDict()
        self.layers["Affine1"] =  Affine(self.params['W1'], self.params['B1'])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] =  Affine(self.params['W2'], self.params['B2'])
        self.lastLayer = SoftmaxWithLoss()
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)

    def gradient(self,x,t):
        self.loss(x,t)
        dout = self.lastLayer.backward(1) # 
        layers_tmp = list(self.layers.values())
        layers_tmp.reverse()
        for layer in layers_tmp:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['B1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['B2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads

In [80]:
pathname = "cifar-10-batches-py"

In [81]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [82]:
data_arr = []
label_arr = []
DATA_BATCH_NUM = 5
for i in range(DATA_BATCH_NUM):
    data_batch = unpickle(pathname+"/data_batch_{}".format(i+1))
    data_arr.extend(data_batch[b'data'])
    label_arr += data_batch[b'labels']
data_arr = np.array(data_arr)
label_arr = np.array(label_arr)

In [85]:
# Cifar

In [103]:
nn = TwoLayerNeuralNet()

In [104]:
data_arr = []
label_arr = []
DATA_BATCH_NUM = 5
for i in range(DATA_BATCH_NUM):
    data_batch = unpickle(pathname+"/data_batch_{}".format(i+1))
    data_arr.extend(data_batch[b'data'])
    label_arr += data_batch[b'labels']
data_arr = np.array(data_arr)
label_arr = np.array(label_arr)

In [105]:
def get_label_one_hot_arr(label_arr,label_num):
    t_batch = np.array([])
    batch_size = len(label_arr)
    for label in label_arr:
        t = np.zeros(label_num)
        t[label] = 1
        t_batch = np.append(t_batch,t)
    return t_batch.reshape(batch_size,label_num)

In [106]:
iters_num = 10000
train_size = data_arr.shape[0]
batch_size = 100
learning_rate = 0.1
loss_arr_cifar = []
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = data_arr[batch_mask]
    t_batch = get_label_one_hot_arr(label_arr[batch_mask],10)
    
    grad = nn.gradient(x_batch,t_batch)
    for key in ("W1","B1","W2","B2"):
        nn.params[key] -= learning_rate * grad[key]
    loss = nn.loss(x_batch,t_batch)
    loss_arr_cifar.append(loss)

In [108]:
loss_arr_cifar[-1]

2.2995503908591974

In [109]:
# Mnist

In [110]:
from dataset.mnist import load_mnist
from collections import OrderedDict
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [111]:
nn_mnist = TwoLayerNeuralNet(input_size=784, hidden_size=50, output_size=10)

In [112]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
loss_arr = []
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = nn.gradient(x_batch,t_batch)
    for key in ("W1","B1","W2","B2"):
        nn_mnist.params[key] -= learning_rate * grad[key]
    loss = nn_mnist.loss(x_batch,t_batch)
    loss_arr.append(loss)


ValueError: shapes (100,784) and (3072,50) not aligned: 784 (dim 1) != 3072 (dim 0)

In [101]:
loss_arr[-1]

14.989828948391237